In [185]:
import geopandas as gpd 
import h3 
import numpy as np
from shapely.geometry import Point, Polygon, MultiPolygon, LineString, mapping
from shapely.ops import split, transform

def amcut(polygon):
    """
    Rough function to handle antimeridian crossings.
    Only works on polygons with lon between -100 and 100.
    """
    x = np.array(polygon.exterior.coords.xy[0])
    y = np.array(polygon.exterior.coords.xy[1])

    if any(i>100 for i in x) and any(i<-100 for i in x):

        idxs_neg = np.where(x<-100)
        x[idxs_neg] += 360

    extended = Polygon(np.array(list(zip(x, y))))
    am = LineString(((180, 90), (180, -90)))

    return MultiPolygon(split(extended, am))

In [215]:
quakes = gpd.read_file('earthquakes1970-2014.json')
quakes.head()

,DateTime,Latitude,Longitude,Depth,Magnitude,MagType,NbStations,Gap,Distance,RMS,Source,EventID,geometry
0,1970-01-04T17:00:40,24.139,102.503,31.0,7.5,Ms,90.0,NaN,NaN,0.0,NEI,1.970010e+09,POINT (102.50300 24.13900)
1,1970-01-06T05:35:51,-9.628,151.458,8.0,6.2,Ms,85.0,NaN,NaN,0.0,NEI,1.970011e+09,POINT (151.45800 -9.62800)
2,1970-01-08T17:12:39,-34.741,178.568,179.0,6.1,Mb,59.0,NaN,NaN,0.0,NEI,1.970011e+09,POINT (178.56800 -34.74100)
3,1970-01-10T12:07:08,6.825,126.737,73.0,6.1,Mb,91.0,NaN,NaN,0.0,NEI,1.970011e+09,POINT (126.73700 6.82500)
4,1970-01-16T08:05:39,60.280,-152.660,85.0,6.0,ML,0.0,NaN,NaN,NaN,AK,NaN,POINT (-152.66000 60.28000)


In [216]:
def flip(x, y):
    return y, x

def get_hex_boundary(point, res):
    hex = h3.geo_to_h3(point.y, point.x, res)
    hex_boundary = h3.h3_to_geo_boundary(hex, geo_json=True)
    return Polygon(hex_boundary)

def get_hex_point(point, res):
    hex = h3.geo_to_h3(point.y, point.x, res)
    hex_boundary = h3.h3_to_geo(hex)
    return transform(flip, Point(hex_boundary))

def get_hex(point, res):
    return h3.geo_to_h3(point.y, point.x, res)

In [217]:
quakes['hex'] = [get_hex(quakes.geometry.loc[x], 4) for x in quakes.index]
quakes.geometry = quakes.geometry.apply(get_hex_boundary, args=[4])
quakes.head()

,DateTime,Latitude,Longitude,Depth,Magnitude,MagType,NbStations,Gap,Distance,RMS,Source,EventID,geometry,hex
0,1970-01-04T17:00:40,24.139,102.503,31.0,7.5,Ms,90.0,NaN,NaN,0.0,NEI,1.970010e+09,"POLYGON ((102.51663 24.01123, 102.54206 24.250...",8440627ffffffff
1,1970-01-06T05:35:51,-9.628,151.458,8.0,6.2,Ms,85.0,NaN,NaN,0.0,NEI,1.970011e+09,"POLYGON ((151.33003 -9.34467, 151.28477 -9.579...",849cb4bffffffff
2,1970-01-08T17:12:39,-34.741,178.568,179.0,6.1,Mb,59.0,NaN,NaN,0.0,NEI,1.970011e+09,"POLYGON ((178.75128 -35.02690, 178.70729 -34.7...",84ba237ffffffff
3,1970-01-10T12:07:08,6.825,126.737,73.0,6.1,Mb,91.0,NaN,NaN,0.0,NEI,1.970011e+09,"POLYGON ((126.69525 6.87740, 126.65669 6.62760...",8468591ffffffff
4,1970-01-16T08:05:39,60.280,-152.660,85.0,6.0,ML,0.0,NaN,NaN,NaN,AK,NaN,"POLYGON ((-152.94990 60.59315, -153.35114 60.4...",840c50dffffffff


In [218]:
quakes.geometry = quakes.geometry.apply(amcut)

In [219]:
quakes['count'] = quakes.groupby(["hex"]).hex.transform('count')
quakes.head()

,DateTime,Latitude,Longitude,Depth,Magnitude,MagType,NbStations,Gap,Distance,RMS,Source,EventID,geometry,hex,count
0,1970-01-04T17:00:40,24.139,102.503,31.0,7.5,Ms,90.0,NaN,NaN,0.0,NEI,1.970010e+09,"MULTIPOLYGON (((102.51663 24.01123, 102.27060 ...",8440627ffffffff,1
1,1970-01-06T05:35:51,-9.628,151.458,8.0,6.2,Ms,85.0,NaN,NaN,0.0,NEI,1.970011e+09,"MULTIPOLYGON (((151.33003 -9.34467, 151.55671 ...",849cb4bffffffff,2
2,1970-01-08T17:12:39,-34.741,178.568,179.0,6.1,Mb,59.0,NaN,NaN,0.0,NEI,1.970011e+09,"MULTIPOLYGON (((178.75128 -35.02690, 178.53527...",84ba237ffffffff,1
3,1970-01-10T12:07:08,6.825,126.737,73.0,6.1,Mb,91.0,NaN,NaN,0.0,NEI,1.970011e+09,"MULTIPOLYGON (((126.69525 6.87740, 126.92837 6...",8468591ffffffff,5
4,1970-01-16T08:05:39,60.280,-152.660,85.0,6.0,ML,0.0,NaN,NaN,NaN,AK,NaN,"MULTIPOLYGON (((-152.94990 60.59315, -152.5560...",840c50dffffffff,2


In [80]:
hexes = []
for lat in range(-80, 80, 2):
    hexes.append(h3.geo_to_h3(lat, 180, 2))

In [81]:
hexes_geo = [Polygon(h3.h3_to_geo_boundary(x, geo_json=True)) for x in hexes]

In [82]:
hex = gpd.GeoDataFrame(hexes_geo, columns=['geometry'], crs='epsg:4326')

In [83]:
hex_amcut = hex.copy()
hex_amcut.geometry = hex_amcut.geometry.apply(amcut)

In [84]:
hex_amcut.head()

,geometry
0,"MULTIPOLYGON (((180.000 -82.091, 179.493 -82.2..."
1,"MULTIPOLYGON (((180.000 -79.062, 178.608 -79.5..."
2,"MULTIPOLYGON (((180.000 -76.194, 178.097 -77.0..."
3,"MULTIPOLYGON (((180.000 -73.472, 180.326 -73.3..."
4,"MULTIPOLYGON (((180.000 -70.823, 180.271 -70.6..."


In [150]:
import cartopy.crs as ccrs
import hvplot.pandas

proj = ccrs.Robinson(150)

In [220]:
quakes_reduced = quakes.groupby('hex').first()[['geometry', 'count']]
quakes_reduced.head(10)

,geometry,count
hex,,
8400645ffffffff,"MULTIPOLYGON (((-1.29579 79.96593, -2.63395 79...",1
8400dc3ffffffff,"MULTIPOLYGON (((98.63353 84.90740, 99.35784 85...",1
840150dffffffff,"MULTIPOLYGON (((18.26882 76.74445, 17.34019 76...",1
8401591ffffffff,"MULTIPOLYGON (((2.43893 79.50282, 1.16819 79.5...",1
84015c3ffffffff,"MULTIPOLYGON (((7.03817 78.37628, 5.91047 78.4...",1
8401831ffffffff,"MULTIPOLYGON (((54.78917 73.06888, 54.38314 73...",2
8401b3bffffffff,"MULTIPOLYGON (((53.84221 70.56687, 53.48301 70...",2
84021d7ffffffff,"MULTIPOLYGON (((-107.98807 76.90446, -106.9995...",1
840263bffffffff,"MULTIPOLYGON (((-72.98820 75.60510, -72.33151 ...",1


In [221]:
types = set()
for geom in quakes_reduced.geometry:
    types.update([type(geom)])
types

{shapely.geometry.multipolygon.MultiPolygon}

In [224]:
# plot = hex_amcut.hvplot(geo=True, fill_color='red', fill_alpha=.8, projection=proj, project=True, global_extent=True, coastline=True, frame_width=1800)
plot = quakes_reduced.hvplot(geo=True, c='count', cmap='RdYlBu_r', line_color='white', line_width=.01, projection=proj, project=True, global_extent=True, coastline=True, frame_width=1800)
plot

/Users/benhuff/opt/miniconda3/envs/h3viz/lib/python3.8/site-packages/geoviews/operation/projection.py:79: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  polys = [g for g in geom if g.area > 1e-15]
/Users/benhuff/opt/miniconda3/envs/h3viz/lib/python3.8/site-packages/geoviews/operation/projection.py:79: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  polys = [g for g in geom if g.area > 1e-15]


:Overlay
   .Polygons.I  :Polygons   [Longitude,Latitude]   (count)
   .Coastline.I :Feature   [Longitude,Latitude]